# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""39614394611904…","""98974952859542…",1.2927e19,"""../../../../da…","""12625860503633…","""37117717968132…",false
"""15234038173069…","""88308720592055…",1.4856e19,"""../../../../da…","""12625860503633…","""16155238513302…",true
"""49969424881575…","""13483422405507…",5.6458e17,"""../../../../da…","""58899174680096…","""37117717968132…",false
"""15506395878320…","""88308720592055…",4.0723e18,"""../../../../da…","""16383145376075…","""37117717968132…",false
"""57859482805389…","""98974952859542…",1.4677e19,"""../../../../da…","""16383145376075…","""38134894516732…",true
"""17666424399408…","""13483422405507…",1.6330e19,"""../../../../da…","""19910495316176…",null,false
"""69660566944120…","""13483422405507…",8.7675e18,"""../../../../da…","""16383145376075…","""38134894516732…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""24906287822007…","""41923957743614…",1.2927e19,"""../../../../da…","""FirstMortgage3…","""10694444699049…","""Rejected"""
"""16920450330872…","""18372075322930…",1.4856e19,"""../../../../da…","""FirstMortgage3…","""10711748221210…","""Accepted"""
"""17503097617606…","""81044596674439…",5.6458e17,"""../../../../da…","""MoneyMarketSav…","""10694444699049…","""Rejected"""
"""12913749763954…","""18372075322930…",4.0723e18,"""../../../../da…","""BasicChecking""","""10694444699049…","""Rejected"""
"""82844023519525…","""41923957743614…",1.4677e19,"""../../../../da…","""BasicChecking""","""28585038371934…","""Accepted"""
"""32506535932083…","""81044596674439…",1.6330e19,"""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""74000969213362…","""81044596674439…",8.7675e18,"""../../../../da…","""BasicChecking""","""28585038371934…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""82587065948713…","""21425229165987…",1.2927e19,"""../../../../da…","""FirstMortgage3…","""17461492197842…","""Rejected"""
"""12449034186719…","""18305619675034…",1.4856e19,"""../../../../da…","""FirstMortgage3…","""14240636017869…","""Accepted"""
"""14680153896357…","""41123443433542…",5.6458e17,"""../../../../da…","""MoneyMarketSav…","""17461492197842…","""Rejected"""
"""10030003527024…","""18305619675034…",4.0723e18,"""../../../../da…","""BasicChecking""","""17461492197842…","""Rejected"""
"""15736682729849…","""21425229165987…",1.4677e19,"""../../../../da…","""BasicChecking""","""49072358533906…","""Accepted"""
"""22637276770428…","""41123443433542…",1.6330e19,"""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""13470607066337…","""41123443433542…",8.7675e18,"""../../../../da…","""BasicChecking""","""49072358533906…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_CLV=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""12974584836859…","""10124454860364…",1.2927e19,"""FirstMortgage3…","""12878186021907…","""Rejected"""
"""20131286213001…","""17267845986325…",1.4856e19,"""FirstMortgage3…","""16258678085361…","""Accepted"""
"""46531680358044…","""10162716038494…",5.6458e17,"""MoneyMarketSav…","""12878186021907…","""Rejected"""
"""48239023461650…","""17267845986325…",4.0723e18,"""BasicChecking""","""12878186021907…","""Rejected"""
"""91613172251530…","""10124454860364…",1.4677e19,"""BasicChecking""","""15172295135836…","""Accepted"""
"""36007618774920…","""10162716038494…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""45222118156308…","""10162716038494…",8.7675e18,"""BasicChecking""","""15172295135836…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'